In [258]:
# encoding=utf-8
import sys
import pylab as pl
from collections import defaultdict,Counter,OrderedDict
import pandas as pd
import numpy as np
# http://www.jb51.net/article/75178.htm
from sklearn import datasets

# 经纬度计算
#计算经纬度坐标距离（米），参考：http://blog.csdn.net/vernice/article/details/46581361
from math import radians, cos, sin, asin, sqrt,pi

def quickDistance(lon1,lat1,lon2,lat2,max_diff):
    # 快速判断两点距离是否超过上限,经纬度差超过max_diff时，直接判定
    if abs( lon1 - lon2 )>max_diff or abs( lat1 - lat2 )>max_diff:
        return 1
    else:
        return 0
    
def calDistance(lon1, lat1, lon2, lat2): 
    """ 
    计算任意两点间的球面距离(m),传参：经度1，纬度1，经度2，纬度2 （十进制度数）  
    """  
    # 转成float类型
    try:
        lon1 = float(lon1)
        lat1 = float(lat1)
        lon2 = float(lon2)
        lat2 = float(lat2)
    except Exception,err:
        print >> sys.stderr,'[ERROR] 传参错误！,非数值形式,退出'
        return -1
    # 空值判断
    if lon1 == 0.0 or lat1 == 0.0 or lon2 == 0.0 or lat2 == 0.0:
        print >>sys.stderr,'[ERROR] 取值为0，非法，退出'
        return -1
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000 

# 读取数据 D:\work\用户建模画像\家公司挖掘\code\warren.xls
print '开始读取数据'
# 数据格式：time lon lat adcode geohash
#df = pd.read_excel('C:\Users\warren\Desktop\warren.xlsx')
#df = pd.read_excel('C:\Users\warren\Desktop\warren1.xlsx')
df = pd.read_csv('C:\Users\warren\Desktop\demo.csv')
#df = pd.read_csv('C:\Users\warren\Desktop\\tmp_location_test1.csv')
#df.head()
print '提取经纬度信息'
# 如果数据量超长，截断
max_count = 1000
start = 0
if len(df)>max_count:
    start = max_count
#df1= df[df.uid=='BA1C6537-599E-4CBD-8E0C-7E183C4B7450']
data = df.iloc[:start,[1,2,3]] # 提取经纬度信息
#data = df.iloc[:,[1,2]] # 提取经纬度信息
print '数据点信息：编号、lon、lat、geohash\n',data
#exit()
points = data.values.tolist()
#data = datasets.load_iris()
#print data.data
#points = [ [i[0],i[1]] for i in data.data[:,:]]
#x=raw_input()
#points = [[int(eachpoint.split("#")[0]), int(eachpoint.split("#")[1])] for eachpoint in open("points","r")]
#==========DBSCAN开始===========
# （1）计算每个数据点相邻的数据点
#      邻域定义为以该点为中心以边长为2*EPs的网格——右三角矩阵
Eps = 200 # 100m的领域
#Eps = 0.28
#dist_dict = {}# 距离字典,2->3:2.331,key为组合字符串
# surroundPoints记录每个点的领域集，对称矩阵
surroundPoints = defaultdict(list) # defualt工厂函数，每定义一个key，就会对应生成默认的list类型空值
for idx1,point1 in enumerate(points):
    for idx2,point2 in enumerate(points):
        if (idx1 < idx2):
            #tmp_dist = sqrt((point1[0]-point2[0])*(point1[0]-point2[0])+(point1[1]-point2[1])*(point1[1]-point2[1]))
            tmp_dist = calDistance(point1[0],point1[1],point2[0],point2[1])
            #dist_dict['%s->%s'] = calDistance(point1[0],point1[1],point2[0],point2[1])
            #dist_dict['%s->%s'%(idx1,idx2)] = tmp_dist
            if(tmp_dist<=Eps):
                surroundPoints[idx1].append(idx2)
                surroundPoints[idx2].append(idx1)
# （2）计算核心点：定义邻域内相邻的数据点的个数大于4的为核心点
#MinPts = 5
MinPts = 3
corePointIdx = [pointIdx for pointIdx,surPointIdxs in surroundPoints.iteritems() if len(surPointIdxs)>=MinPts]
# （3）计算边界点：邻域内包含某个核心点的非核心点，定义为边界点
borderPointIdx = []
for pointIdx,surPointIdxs in surroundPoints.iteritems():
    if (pointIdx not in corePointIdx):
        for onesurPointIdx in surPointIdxs:
            if onesurPointIdx in corePointIdx:
                borderPointIdx.append(pointIdx)
                break
# （4）识别噪音：噪音点既不是边界点也不是核心点
noisePointIdx = [pointIdx for pointIdx in range(len(points)) if pointIdx not in corePointIdx and pointIdx not in borderPointIdx]
# 核心点、边界点、噪音点
#corePoint = [points[pointIdx] for pointIdx in corePointIdx] 
#borderPoint = [points[pointIdx] for pointIdx in borderPointIdx]
#noisePoint = [points[pointIdx] for pointIdx in noisePointIdx]
groups = [idx for idx in range(len(points))]
# （5）聚类:①先处理核心点②并入边界点
# ①各个核心点与其邻域内的所有核心点放在同一个簇中
for pointidx,surroundIdxs in surroundPoints.iteritems():
    for oneSurroundIdx in surroundIdxs:
        if (pointidx in corePointIdx and oneSurroundIdx in corePointIdx and pointidx < oneSurroundIdx):
            # 将编号大的节点数值改成同类小编号
            for idx in range(len(groups)):
                if groups[idx] == groups[oneSurroundIdx]:
                    groups[idx] = groups[pointidx]
# ②边界点跟其邻域内的某个核心点放在同一个簇中
for pointidx,surroundIdxs in surroundPoints.iteritems():
    for oneSurroundIdx in surroundIdxs:
        if (pointidx in borderPointIdx and oneSurroundIdx in corePointIdx):
            # 边界节点的存储的数值改成邻近核心节点编号
            groups[pointidx] = groups[oneSurroundIdx]
            break
# 
print '邻接矩阵：'
print '\n'.join(['\t'.join([str(i)]+[str(j) for j in v]) for i,v in surroundPoints.iteritems()])
print '核心点集：',corePointIdx
print '边界点集：',borderPointIdx
print '噪声点集：',noisePointIdx
print '聚类结果：',groups
#print '\n'.join(['\t'.join([[str(i),str(j)] for i,j in enumerate(groups)])])

out = [[k,v] for k,v in Counter(groups).iteritems() if v > 1]
print out
# 计算聚类簇分组
result = []
for i in range(len(groups)):
    if i in noisePointIdx:
        # 忽略噪声点
        continue
    if i == groups[i]:
        # 类中第一个节点，格式:[中心点,数目,成员点,密度,平均点,平均距离]
        result.append([i,1,[i],-1,[],-1])
    elif i > groups[i]:
        # 类中后续节点
        for j in range(len(result)):
            if result[j][0] == groups[i]:
                result[j][1] += 1
                result[j][2].append(i)
                break
    else: # i < groups[i]不存在
        pass
result = filter(lambda x:x[1]>1,result) # 过滤
#import copy
#result = copy.deepcopy(result_raw)
result.sort(key=lambda c: c[1],reverse=True) # 按照类节点多少排序
cluster_num = len(result)
#print '聚类分组：','\n'.join(['\t'.join([repr(j) for j in i]) for i in result])
# 计算各类中心点及密度、平均距离
for i in range(len(result)):
    xy_list = [[j]+points[j] for j in result[i][2]] # [id lon lat]
    num = len(xy_list)
    x_min = min([j[1] for j in xy_list])
    x_max = max([j[1] for j in xy_list])
    y_min = min([j[2] for j in xy_list])
    y_max = max([j[2] for j in xy_list])
    x_avg = sum(([j[1] for j in xy_list]))/num
    y_avg = sum(([j[2] for j in xy_list]))/num
    # 找代表点（离质心近，且代表大多数点，距离并非首要因素，平均距离最小，消除噪声点的影响）
    dist_list = [] # 质心到各点的距离
    dist_avg_list = [] # 各点作为中心点时的簇平均距离
    dist_min = calDistance(x_min,y_min,x_max,y_max) # 与平均点最小距离初始化为矩形框对角线
    dist_i = xy_list[0][0] # 与质心最近的点编号，初始化为第一个点
    for j in range(num):
        cur_dist = calDistance(x_avg,y_avg,xy_list[j][1],xy_list[j][2])
        dist_list.append(cur_dist)
        if cur_dist <= dist_min:
            dist_min = cur_dist
            #dist_i = xy_list[j][0]
        # 计算到其它点的平均距离
        cur_dist_sum = 0
        for k in range(num):
            if k == j:
                continue
            '''
            elif k < j:
                cur_dist_sum += dist_dict['%s->%s'%(xy_list[k][0],xy_list[j][0])]
            else: # k>j
                cur_dist_sum += dist_dict['%s->%s'%(xy_list[j][0],xy_list[k][0])]
            '''
            # 时间换空间,每次都计算距离,不再使用字典
            cur_dist_sum += calDistance(xy_list[k][1],xy_list[k][2],xy_list[j][1],xy_list[j][2])
        dist_avg_list.append(cur_dist_sum/(num-1))
    print >>sys.stdout,'平均距离：%s'%(repr(dist_avg_list))
    # 计算簇最小平均距离
    dist_i = xy_list[dist_avg_list.index(min(dist_avg_list))][0]
    # 计算平均距离
    dist_avg = sum(dist_list)/num
    # 计算密度(欧氏距离近似)
    area = (x_max - x_min)*(y_max - y_min)
    if area <= 0:
        print >>sys.stderr,'面积计算出错，找不到矩形框,x=[%s,%s],y=[%s,%s]'%(x_min,x_max,y_min,y_max)
        phou = -1
    else:
        phou = result[i][1]/area # 计算簇密度
    result[i][0] = dist_i
    result[i][3] = phou
    result[i][4] = [x_avg,y_avg]
    result[i][5] = dist_avg
centerPointsIdx = [i[0] for i in result]
avgPoints = [i[4] for i in result]
print '%s个点包含%s个噪声点,%s个核心点,%s个边界点,最后一共分成%s类'%(len(groups),len(noisePointIdx),len(corePointIdx),len(borderPointIdx),cluster_num)
print '聚类分组：','\n'.join(['\t'.join([repr(j) for j in i]) for i in result])
print '中心点：'
print centerPointsIdx
print '质心点：'
print avgPoints
print '#'*100
#################组装聚类结果########################
# 顶层结构：[uid all cluster_num cluster_info]
# cluster_info结构：\001分隔，每组格式：[id num density center_point centroid_point avg_dist point_list]
out_list = ['uid',len(points),cluster_num,'-']
cluster_list = []
for i in range(cluster_num):
    # 每个类格式：[中心点,数目,成员点,密度,平均点,平均距离]
    cluster_list.append([str(i+1),str(result[i][1]),'%.3f'%(result[i][3]),'%s,%s,%s'%(points[result[i][0]][0],points[result[i][0]][1],points[result[i][0]][2]),'%s,%s'%(result[i][4][0],result[i][4][1]),'%.3f'%(result[i][5]),'&'.join(['%s,%s,%s'%(points[j][0],points[j][1],points[j][2]) for j in result[i][2]])])
out_list[-1] = '\001'.join(['\002'.join(i) for i in cluster_list])
print out_list
#############################可视化部分#####################################
# json形式输出，js页面可视化
import json
tmp_data = data.values.tolist()
out = []
for i in tmp_data:
    out.append(dict(zip(['lng','lat'],i)))
out_str = json.dumps(out,indent=4)
import re
f=open(r"C:\Users\warren\Desktop\cluster_update.html", "r+")
open('C:\Users\warren\Desktop\cluster_update1.html', 'w').write(re.sub(r'\$data', out_str, f.read()))
import webbrowser
webbrowser.open('C:\Users\warren\Desktop\cluster_update1.html')
# 附：高德地图api开发指南；http://www.cnblogs.com/milkmap/p/3707711.html

#"""
# 绘制各个聚类簇,按照形状、颜色依次组合
colors = ['r','b','g','w','y','c','m'];shapes = ['o','s','*','d','p','^','v','1','2','3','4']
for i in range(cluster_num):
    # 中心点编号,数目,成员,密度,平均点
    pl.plot([points[j][0] for j in result[i][2]],[points[j][1] for j in result[i][2]],'%s%s'%(shapes[i/len(shapes)],colors[i%len(colors)]))
    # [id num density dist]
    pl.annotate('%s,%s,%.2f,%.2f'%(i+1,result[i][1],result[i][3],result[i][5]), xy=result[i][4], xytext=result[i][4],arrowprops=dict(facecolor='blue', shrink=0.03))
    #pl.annotate('cluster:%s,%s,%s'%(i,result[i][1],result[i][3]), xy=result[i][4], xytext=result[i][4]+0.5,arrowprops=dict(facecolor='blue', shrink=0.03))
# 打印中心点,平均点
pl.plot([points[i][0] for i in centerPointsIdx], [points[i][1] for i in centerPointsIdx], 'xc',label='0',markersize=20)
pl.plot([i[0] for i in avgPoints], [i[1] for i in avgPoints], '+m',label='0',markersize=20)
# 打印噪音点，黑色
pl.plot([eachpoint[0] for eachpoint in noisePoint], [eachpoint[1] for eachpoint in noisePoint], 'ok')  
pl.xlabel('X');pl.ylabel('Y')
pl.title('DBSCAN Cluster Result:%s'%(cluster_num))
pl.grid()
pl.show()
    


开始读取数据
提取经纬度信息
数据点信息：编号、lon、lat、geohash
            lon        lat       geohash
0    116.504473  39.921454  wx4g557zqfrm
1    116.504303  39.921452  wx4g557z7409
2    116.504531  39.921445  wx4g55kp22dx
3    116.504337  39.921517  wx4g557zey4h
4    116.504476  39.921463  wx4g557zrh1d
5    116.504473  39.921454  wx4g557zqfrm
6    116.504473  39.921454  wx4g557zqfrm
7    116.504476  39.921463  wx4g557zrh1d
8    116.504476  39.921463  wx4g557zrh1d
9    116.504301  39.921528  wx4g557zfbpn
10   116.504301  39.921528  wx4g557zfbpn
11   116.504532  39.921411  wx4g55kp067k
12   116.504476  39.921463  wx4g557zrh1d
13   116.504567  39.921441  wx4g55kp1pvx
14   116.504451  39.921477  wx4g557zqqxn
15   116.504301  39.921528  wx4g557zfbpn
16   116.504476  39.921463  wx4g557zrh1d
17   116.504567  39.921441  wx4g55kp1pvx
18   116.504337  39.921517  wx4g557zey4h
19   116.504473  39.921454  wx4g557zqfrm
20   116.504476  39.921463  wx4g557zrh1d
21   116.504301  39.921528  wx4g557zfbpn
22   116.504543  

In [105]:
import pylab as pl
from collections import defaultdict,Counter
import pandas as pd
import numpy as np

# 读取数据 D:\work\用户建模画像\家公司挖掘\code\warren.xls
print '开始读取数据'
# 数据格式：time lon lat adcode geohash
df = pd.read_excel('C:\Users\warren\Desktop\warren.xlsx')
#df.head()
print '提取经纬度信息'
data = df.iloc[:,[1,2]] # 提取经纬度信息
# json形式输出，js页面可视化
import json
tmp_data = data.values.tolist()
out = []
for i in tmp_data:
    out.append(dict(zip(['lng','lat'],i)))
out_str = json.dumps(out,indent=4)
import re
f=open(r"C:\Users\warren\Desktop\cluster_update.html", "r+")
open('C:\Users\warren\Desktop\cluster_update1.html', 'w').write(re.sub(r'\$data', out_str, f.read()))


开始读取数据
提取经纬度信息


In [48]:
import sys
import math

# 经纬度计算
def calculateDistance(lon1,lat1,lon2,lat2):
    # 计算两个坐标点之间的距离 (lon1,lat1),(lon2,lat2)
    print locals()
    _earth_r = 6371.393 # 地球半径
    _pai = 3.1415926 # π值
    # 类型转换
    for var in ('lon1','lat1','lon2','lat2'):
        # 类型转换
        if type(eval(var)) != type(0.0):
            try:
                tmp = eval('float(var)')
                eval(var+'='+tmp)
                #eval(var + '='+ 'float(' + var + ')')
                #eval('%s=float(%s)'%(var))
                #eval(var) = float(eval(var))
                #eval('var = double(var)',locals())
            except Exception,err:
                print >> sys.stderr,'[ERROR] 传参错误！(%s),%s非数值形式,退出'%(err,var)
                return 0
        # 空值判断
        if eval(var) == 0.0:
            print >>sys.stderr,'[ERROR] %s取值为0，非法，退出'%(var)
            return 0
    # 计算球面距离
    #dist = _earth_r*Math.acos(Math.sin(x1*pai/180)*Math.sin(x2*pai/180)+Math.cos(x1*pai/180)*Math.cos(x2*pai/180)*Math.cos(y1*pai/180-y2*pai/180));
    dist = _earth_r*math.acos(math.sin(lat1*_pai/180)*math.sin(lat2*_pai/180)+math.cos(lat1*_pai/180)*math.cos(lat2*_pai/180)*math.cos(lon1*_pai/180-lon2*_pai/180))
    return dist

if __name__ == '__main__':
    calculateDistance("30.579249", "103.959226", "tes0.578909", "103.958961") # 非法取值
    calculateDistance("30.579249", "103.959226", "0.0", "103.958961") # 一个空值
    calculateDistance("30.579249", "103.959226", "30.579249", "103.959226") # 两个坐标相同
    calculateDistance("30.579249", "103.959226", "30.578909", "103.958961")
    calculateDistance("30.579249", "103.959226", "30.636895", "104.029880")
    calculateDistance("40.0", "116.5", "31.0", "121.5")
    

    

{'lon1': '30.579249', 'lon2': 'tes0.578909', 'lat1': '103.959226', 'lat2': '103.958961'}
{'lon1': '30.579249', 'lon2': '0.0', 'lat1': '103.959226', 'lat2': '103.958961'}
{'lon1': '30.579249', 'lon2': '30.579249', 'lat1': '103.959226', 'lat2': '103.959226'}
{'lon1': '30.579249', 'lon2': '30.578909', 'lat1': '103.959226', 'lat2': '103.958961'}
{'lon1': '30.579249', 'lon2': '30.636895', 'lat1': '103.959226', 'lat2': '104.029880'}
{'lon1': '40.0', 'lon2': '31.0', 'lat1': '116.5', 'lat2': '121.5'}


[ERROR] 传参错误！(could not convert string to float: lon1),lon1非数值形式,退出
[ERROR] 传参错误！(could not convert string to float: lon1),lon1非数值形式,退出
[ERROR] 传参错误！(could not convert string to float: lon1),lon1非数值形式,退出
[ERROR] 传参错误！(could not convert string to float: lon1),lon1非数值形式,退出
[ERROR] 传参错误！(could not convert string to float: lon1),lon1非数值形式,退出
[ERROR] 传参错误！(could not convert string to float: lon1),lon1非数值形式,退出


In [197]:
import sys
from math import *

# 经纬度计算——注：有问题，个别ｃａｓｅ分母为０，无法运行
def calDistance(lon1,lat1,lon2,lat2):
    # 计算两个坐标点之间的距离 (lon1,lat1),(lon2,lat2)
    # 代码参考：http://www.cnblogs.com/yejingcn/p/4537863.html?utm_source=tuicool&utm_medium=referral
    #print locals()
    # 类型转换
    try:
        lon1 = float(lon1)
        lat1 = float(lat1)
        lon2 = float(lon2)
        lat2 = float(lat2)
    except Exception,err:
        print >> sys.stderr,'[ERROR] 传参错误！,非数值形式,退出'
        return -1
    # 空值判断
    if lon1 == 0.0 or lat1 == 0.0 or lon2 == 0.0 or lat2 == 0.0:
        print >>sys.stderr,'[ERROR] 取值为0，非法，退出'
        return -1
    # 计算球面距离
    #dist = _earth_r*Math.acos(Math.sin(x1*pai/180)*Math.sin(x2*pai/180)+Math.cos(x1*pai/180)*Math.cos(x2*pai/180)*Math.cos(y1*pai/180-y2*pai/180));
    #dist = _earth_r*math.acos(math.sin(lat1*_pai/180)*math.sin(lat2*_pai/180)+math.cos(lat1*_pai/180)*math.cos(lat2*_pai/180)*math.cos(lon1*_pai/180-lon2*_pai/180))
    ra = 6378.140  # 赤道半径 (km)
    rb = 6356.755  # 极半径 (km)
    flatten = (ra - rb) / ra  # 地球扁率
    rad_lat_A = radians(lat1)
    rad_lng_A = radians(lon1)
    rad_lat_B = radians(lat2)
    rad_lng_B = radians(lon2)
    pA = atan(rb / ra * tan(rad_lat_A))
    pB = atan(rb / ra * tan(rad_lat_B))
    xx = acos(sin(pA) * sin(pB) + cos(pA) * cos(pB) * cos(rad_lng_A - rad_lng_B))
    c1 = (sin(xx) - xx) * (sin(pA) + sin(pB)) ** 2 / cos(xx / 2) ** 2
    c2 = (sin(xx) + xx) * (sin(pA) - sin(pB)) ** 2 / sin(xx / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (xx + dr)
    return distance

if __name__ == '__main__':
    print calDistance("30.579249", "103.959226", "tes0.578909", "103.958961") # 非法取值
    print calDistance("30.579249", "103.959226", "0.0", "103.958961") # 一个空值
    print calDistance("30.579249", "103.959226", "30.579249", "103.959226") # 两个坐标相同
    print calDistance("30.579249", "103.959226", "30.578909", "103.958961")
    print calDistance("30.579249", "103.959226", "30.636895", "104.029880")
    print calDistance("40.0", "116.5", "31.0", "121.5")
    print calDistance('118.796877','32.060255','116.407395','39.904211') # 南京到北京的距离896.533 km
    

    

-1
-1
9.50416922569e-05
0.0309671075332
8.03913647571
738.614952468
896.532784703


[ERROR] 传参错误！,非数值形式,退出
[ERROR] 取值为0，非法，退出


In [4]:
#计算经纬度坐标距离，参考：http://blog.csdn.net/vernice/article/details/46581361
import sys
from math import radians, cos, sin, asin, sqrt ,pi 
#import math
  
def calDistance(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 转成float类型
    try:
        lon1 = float(lon1)
        lat1 = float(lat1)
        lon2 = float(lon2)
        lat2 = float(lat2)
    except Exception,err:
        print >> sys.stderr,'[ERROR] 传参错误！,非数值形式,退出'
        return -1
    # 空值判断
    if lon1 == 0.0 or lat1 == 0.0 or lon2 == 0.0 or lat2 == 0.0:
        print >>sys.stderr,'[ERROR] 取值为0，非法，退出'
        return -1
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000 

if __name__ == '__main__':
    print calDistance("30.579249", "103.959226", "tes0.578909", "103.958961") # 非法取值
    print calDistance("30.579249", "103.959226", "0.0", "103.958961") # 一个空值
    print calDistance("30.579249", "103.959226", "30.579249", "103.959226") # 两个坐标相同
    print calDistance("30.579249", "103.959226", "30.578909", "103.958961")
    print calDistance("30.579249", "103.959226", "30.636895", "104.029880")
    print calDistance("40.0", "116.5", "31.0", "121.5")
    print calDistance('118.796877','32.060255','116.407395','39.904211') # 南京到北京的距离896.533 km
    R = 6371000
    print 0.0018*pi*R/180

-1
-1
0.0
30.8457080039
8007.82935937
736563.355211
898211.728891
200.15086796


[ERROR] 传参错误！,非数值形式,退出
[ERROR] 取值为0，非法，退出


In [244]:
groups = [0, 1, 1, 1, 4, 5, 1, 7, 8, 9, 1, 11, 1, 13, 1, 7, 16, 7, 18, 1, 20]
#print noisePointIdx
# 计算聚类簇分组
result = []
for i in range(len(groups)):
    #if i in noisePointIdx:
    #    # 忽略噪声点
    #    continue
    if i == groups[i]:
        # 类中第一个节点，格式:[中心点,数目,成员点,密度,平均点,平均距离]
        result.append([i,1,[i]])
    elif i > groups[i]:
        # 类中后续节点
        for j in range(len(result)):
            if result[j][0] == groups[i]:
                result[j][1] += 1
                result[j][2].append(i)
                break
    else: # i < groups[i]不存在
        pass
print result
result= filter(lambda x:x[1]>1,result)
print result

[[0, 1, [0]], [1, 8, [1, 2, 3, 6, 10, 12, 14, 19]], [4, 1, [4]], [5, 1, [5]], [7, 3, [7, 15, 17]], [8, 1, [8]], [9, 1, [9]], [11, 1, [11]], [13, 1, [13]], [16, 1, [16]], [18, 1, [18]], [20, 1, [20]]]
[[1, 8, [1, 2, 3, 6, 10, 12, 14, 19]], [7, 3, [7, 15, 17]]]
